### Experimenting and benchmarking DuckDB

Since serving from memory is still too expensive we'll try to use DuckDB in the disk mode, comparing it's performance to that of PostGIS.

Phenomenally, even without spatial indexes DuckDB achieves 1300ms runtime on request, and that's flat across any sizes:

<details>
<summary>Query Profile</summary>

```text
┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││    Query Profiling Information    ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘

┌─────────────────────────────────────┐
│┌───────────────────────────────────┐│
││         Total Time: 1.30s         ││
│└───────────────────────────────────┘│
└────────────────────────��────────────┘
┌─────────────────────────────────────┐
│          Optimizer: 0.0001s         │
│┌───────────────────────────────────┐│
││      Column Lifetime: 0.0000s     ││
││     Common Aggregate: 0.0000s     ││
││   Common Subexpressions: 0.0000s  ││
││Compressed Materialization: 0.0000s││
││        Deliminator: 0.0000s       ││
││     Duplicate Groups: 0.0000s     ││
││    Expression Rewriter: 0.0000s   ││
││         Extension: 0.0000s        ││
││       Filter Pullup: 0.0000s      ││
││      Filter Pushdown: 0.0000s     ││
││         In Clause: 0.0000s        ││
││        Join Order: 0.0000s        ││
││        Regex Range: 0.0000s       ││
││      Reorder Filter: 0.0000s      ││
││  Statistics Propagation: 0.0000s  ││
││           Top N: 0.0000s          ││
││      Unnest Rewriter: 0.0000s     ││
││      Unused Columns: 0.0000s      ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
┌─────────────────────────────────────┐
│      Physical Planner: 0.0001s      │
│┌───────────────────────────────────┐│
││      Column Binding: 0.0000s      ││
││        Create Plan: 0.0001s       ││
││       Resolve Types: 0.0000s      ││
│└───────────────────────────────────┘│
└───────────���─────────────────────────┘
┌─────────────────────────────────────┐
│           Planner: 0.0002s          │
│┌───────────────────────────────────┐│
││          Binder: 0.0002s          ││
│└───────────────────────────────────┘│
└─────────────────────────────────────┘
┌───────────────────────────┐
│      RESULT_COLLECTOR     │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│             0             │
│          (0.00s)          │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│      EXPLAIN_ANALYZE      │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│             1             │
│          (0.00s)          │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│       HASH_GROUP_BY       │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│             #0            │
│          sum(#1)          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│             5             │
│          (0.02s)          │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│         PROJECTION        │
│   ─ ─ ─ �� ─ ─ ─ ─ ─ ─ ─   │
│        type_overlay       │
│  CAST(area_ha AS DOUBLE)  │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│           871618          │
│          (0.00s)          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│      sample_count: 2      │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│ sample_tuples_count: 4096 │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│       total_count: 0      │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│      sample_count: 2      │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│ sample_tuples_count: 4096 │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│       total_count: 0      │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│        0.000000000        │
└─────────────┬─────────────┘                             
┌───────���─────┴─────────────┐
│           FILTER          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│ ST_Contains('\x02\x04\x00 │
│ \x00\x00\x00\x00\x00)\xE2X│
│  \xC1\x7F\x9DDB\x8Bg\xFE@ │
│\x1CkpB\x02\x00\x00\x0...  │
│\x00\x00\x05\x00\x00\x...  │
│ \x00\x00\x00\xA0,\xC0\x02E│
│ \x1C+\xC0\xB0Z\xA9\xE2\xAF│
│\x93H@\xA0,\xC0\x02E\x...  │
│\xE0\xCC\x1D~c\x0DN@\x...  │
│ \xDBI\xF1\xCC\x1F@\xE0\xCC│
│ \x1D~c\x0DN@\xC0\x93\xDBI │
│\xF1\xCC\x1F@\xB0Z\xA9...  │
│ \x93H@\xA0,\xC0\x02E\x1C+ │
│\xC0\xB0Z\xA9\xE2\xAF\...  │
│ :GEOMETRY, ST_GeomFromText│
│        (geometry))        │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│         EC: 174333        │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│           871618          │
│          (5.35s)          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│      sample_count: 2      │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│ sample_tuples_count: 4096 │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│       total_count: 0      │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│        0.000000000        │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│ST_GeomFromText(VARCHAR) ->│
│          GEOMETRY         │
│     40735250.000000000    │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│        0.000000000        │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│         SEQ_SCAN          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│    nfi_awi_overlay_2022   │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│          geometry         │
│        type_overlay       │
│          area_ha          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│         EC: 871667        │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│           871667          │
│          (0.22s)          │
└───────────────────────────┘                             
```
</details>

In [ ]:
import geopandas as gpd

duckDbPath = 'area/wee_forest.duckdb'
datasets = {}

In [ ]:
# Runtime 18m, 4GB final db size

import os
import duckdb

# Connect to the DuckDB database
conn = duckdb.connect('../data/area/wee_forest.duckdb')

# Get a list of all Parquet files in the directory
parquet_files = [f for f in os.listdir('../data/area/') if f.endswith('.parquet')]

# Loop over the files and read each one
for file in parquet_files:
    # Create a table name by removing the file extension
    table_name = os.path.splitext(file)[0]
    
    # Drop the table if it already exists
    conn.execute(f"DROP TABLE IF EXISTS {table_name}")
    
    # Read the Parquet file and create a new table in DuckDB
    conn.execute(f"""
        CREATE TABLE {table_name} AS
        SELECT * FROM parquet_scan('../data/area/{file}')
    """)

    # Get a list of all columns in the table
    columns = conn.execute(f"PRAGMA table_info({table_name})").fetchdf()['name']

    # Create an index for each column
    # It's technically not necessary to index both X an Y as we're searching by Y first due to GB specific shape but having both doesn't hurt too much
    for column in columns:
        conn.execute(f"CREATE INDEX IF NOT EXISTS idx_{table_name}_{column} ON {table_name} ({column})")


for table in parquet_files:
    df = conn.execute(f"SELECT * FROM {os.path.splitext(table)[0]}").fetchdf()
    print(df)

conn.close()

In [ ]:

# Runtime: 3m30s per year for overlay
for year in range(2022, 2023):
    datasets[f'nfi_awi_overlay_{year}'] = gpd.read_file(f'../data/processed/gb_nfi_awi_overlay_{year}.geojson')

In [ ]:
import pandas as pd
dataset = pd.read_parquet('../python/nfi_awi_overlay_2022_points.parquet')

In [ ]:
## Runtime: 8m per year for overlay
import duckdb
import pandas as pd

dataset = pd.read_parquet('../python/nfi_awi_overlay_2022_points.parquet')

# Connect to the DuckDB database
conn = duckdb.connect(duckDbPath)

# Write the GeoDataFrame to the DuckDB database
dataset.to_sql('nfi_awi_overlay_2022_points_sql', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

In [ ]:
## Runtime: 8m per year for overlay
import duckdb
import pandas as pd

dataset = pd.read_parquet('../python/nfi_awi_overlay_2022_points.parquet')

# Connect to the DuckDB database
conn = duckdb.connect(duckDbPath)

conn.execute("""
    CREATE TABLE nfi_awi_overlay_2022_points AS SELECT * FROM parquet_scan('nfi_awi_overlay_2022_points.parquet')
""")

# Close the connection
conn.close()

In [ ]:
import duckdb

conn = duckdb.connect(duckDbPath)
tables = conn.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'main'").fetchall()
print(tables)

In [ ]:
import duckdb
from shapely.geometry import Polygon

# Set up a connection to the database
conn = duckdb.connect(duckDbPath)

xmin, ymin, xmax, ymax = -3.286959553518244,54.43959754873018,5.5518254715021556,55.753058254667735

query = f"""
    SELECT type_overlay, SUM(area_ha) as total_area
    FROM nfi_awi_overlay_2022_points
    WHERE x BETWEEN {xmin} AND {xmax} AND y BETWEEN {ymin} AND {ymax}
    GROUP BY type_overlay
    """

conn.execute("SET profiling_mode='detailed'")
conn.sql(query).explain('analyze')

In [ ]:
import duckdb
from shapely.geometry import Polygon

# Set up a connection to the database
conn = duckdb.connect('wee_forest.duckdb')

# xmin, ymin, xmax, ymax = -3.286959553518244,54.43959754873018,5.5518254715021556,55.753058254667735
xmin, ymin, xmax, ymax = -13.555214010199336,49.15380509633303,7.950139192606855,60.1045987744817 

query = f"""
    SELECT type_overlay, SUM(area_ha) as total_area
    FROM nfi_awi_overlay_2022_points
    WHERE x BETWEEN {xmin} AND {xmax} AND y BETWEEN {ymin} AND {ymax}
    GROUP BY type_overlay
    """

conn.execute("SET profiling_mode='detailed'")
conn.sql(query).explain('analyze')

In [ ]:
import duckdb

# Connect to the DuckDB database# Connect to the DuckDB database in read-only mode
conn = duckdb.connect(database=duckDbPath, read_only=True)

# Get a list of all indexes
indexes = conn.execute("SELECT name FROM sqlite_master WHERE type='index'").fetchall()

# Print all indexes
for index in indexes:
    print(index)

# Close the connection
conn.close()

In [ ]:
import duckdb

# Connect to the DuckDB database
conn = duckdb.connect(duckDbPath)

# Create an index on the 'column_name' column of the 'table_name' table
conn.execute("CREATE INDEX nfi_awi_overlay_2022_points_type_overlay_idx ON nfi_awi_overlay_2022_points (type_overlay)")
conn.execute("CREATE INDEX nfi_awi_overlay_2022_points_type_combined_idx ON nfi_awi_overlay_2022_points (type_combined)")
conn.execute("CREATE INDEX nfi_awi_overlay_2022_points_type_aggregate_idx ON nfi_awi_overlay_2022_points (type_aggregate)")
conn.execute("CREATE INDEX nfi_awi_overlay_2022_points_type_source_idx ON nfi_awi_overlay_2022_points (type_source)")
conn.execute("CREATE INDEX nfi_awi_overlay_2022_points_area_ha_idx ON nfi_awi_overlay_2022_points (area_ha)")
conn.execute("CREATE INDEX nfi_awi_overlay_2022_points_x_idx ON nfi_awi_overlay_2022_points (x)")
conn.execute("CREATE INDEX nfi_awi_overlay_2022_points_y_idx ON nfi_awi_overlay_2022_points (y)")

# Close the connection
conn.close()

In [ ]:
import duckdb

# Connect to the DuckDB database
conn = duckdb.connect(duckDbPath)

# Create an index on the 'column_name' column of the 'table_name' table
conn.execute("CREATE INDEX nfi_awi_overlay_2022_points_sql_type_overlay_idx ON nfi_awi_overlay_2022_points_sql (type_overlay)")
conn.execute("CREATE INDEX nfi_awi_overlay_2022_points_sql_type_combined_idx ON nfi_awi_overlay_2022_points_sql (type_combined)")
conn.execute("CREATE INDEX nfi_awi_overlay_2022_points_sql_type_aggregate_idx ON nfi_awi_overlay_2022_points_sql (type_aggregate)")
conn.execute("CREATE INDEX nfi_awi_overlay_2022_points_sql_type_source_idx ON nfi_awi_overlay_2022_points_sql (type_source)")
conn.execute("CREATE INDEX nfi_awi_overlay_2022_points_sql_area_ha_idx ON nfi_awi_overlay_2022_points_sql (area_ha)")
conn.execute("CREATE INDEX nfi_awi_overlay_2022_points_sql_x_idx ON nfi_awi_overlay_2022_points_sql (x)")
conn.execute("CREATE INDEX nfi_awi_overlay_2022_points_sql_y_idx ON nfi_awi_overlay_2022_points_sql (y)")

# Close the connection
conn.close()

In [ ]:
import duckdb
from shapely.geometry import Polygon

# Set up a connection to the database
conn = duckdb.connect(duckDbPath)

# Install and load the spatial extension
conn.execute("LOAD spatial;")

# xmin, ymin, xmax, ymax = 5.5518254715021556,54.43959754873018,-3.286959553518244,55.753058254667735
xmin, ymin, xmax, ymax = -13.555214010199336,49.15380509633303,7.950139192606855,60.1045987744817 
polygon = Polygon([(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin), (xmin, ymin)])

query = f"""
    SELECT type_overlay, SUM(area_ha) as total_area
    FROM nfi_awi_overlay_2022
    WHERE ST_Contains(
        ST_GeomFromText('SRID=4326;{polygon.wkt}'),
        ST_GeomFromText(geometry)
    )
    GROUP BY type_overlay
    """

# Execute the query and fetch the results
conn.execute("SET profiling_mode='detailed'")
conn.sql(query).explain('analyze')


In [ ]:
# Create an index on the 'geometry' column
conn.execute("CREATE INDEX idx_geometry ON nfi_awi_overlay_2022(geometry)")

In [ ]:
# Set up a connection to the database
conn = duckdb.connect(duckDbPath)

# Get information about the columns of the table
result = conn.execute("PRAGMA table_info('nfi_awi_overlay_2022_points')").fetchall()

# Print the results
for row in result:
    print(row)

# Close the connection
conn.close()

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import duckdb
from shapely.geometry import Polygon
from dotenv import load_dotenv
import os

app = Flask(__name__)
CORS(app)

load_dotenv()

# Set up a connection to the database
conn = duckdb.connect(duckDbPath)

@app.route('/calculate_areas', methods=['GET'])
def calculate_areas():
    type = request.args.get('type')
    bounds = list(map(float, request.args.get('bounds').split(',')))
    xmin, ymin, xmax, ymax = bounds
    polygon = Polygon([(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin), (xmin, ymin)])

    # Create a SQL query
    query = f"""
        SELECT {type}, SUM(area_ha) as total_area
        FROM nfi_awi_overlay_2022
        WHERE ST_Intersects(
            geometry,
            ST_GeomFromText('SRID=4326; {polygon.wkt}')
        )
        GROUP BY {type}
    """

    # Execute the query and fetch the results
    result = conn.execute(query).fetchall()
    keys = ['type', 'total_area']
    areas = {dict(zip(keys, row))[type]: dict(zip(keys, row))['total_area'] for row in result}

    return jsonify(areas)

if __name__ == '__main__':
    app.run(port=3901, debug=False)